# from workflow.py

In [59]:
from copyreg import pickle
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
import requests
import io
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
imputer_num = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
imputer_cat = SimpleImputer(strategy="most_frequent")
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from flytekit import task, workflow
from joblib import dump
from sklearn.preprocessing import OneHotEncoder
from typing import Tuple
import pickle
hi=None
scale=None
from sklearn.preprocessing import MinMaxScaler

In [60]:

def get_dataset() -> pd.DataFrame:
    #return load_digits(as_frame=True).frame
   
    #url = "https://github.com/smadarab/flytelab/raw/main/census.csv" # Make sure the url is the raw version of the file on GitHub
    url="https://github.com/smadarab/flytelab/raw/main/census 2.csv"
    download = requests.get(url).content
    df = pd.read_csv(io.StringIO(download.decode('utf-8')),sep=',')
    print("df is created",df.columns)
    #df.dropna(inplace=True)
    #df = df.reset_index()
    print(df.columns)
    return(df)

def train_model(train: pd.DataFrame) -> Tuple[AdaBoostClassifier,OneHotEncoder]:
    num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loss', 'hours-per-week']
    cat_cols = ['workclass', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']
    log_transform_cols = ['capital-loss', 'capital-gain']    
    def get_cat_cols(X):
        return X[cat_cols]
    def get_num_cols(X):
        return X[num_cols]
    def get_log_transform_cols(X):
        return X[log_transform_cols]
    def get_dummies(X):
        print('\n \n',type(X))
        return pd.get_dummies(pd.DataFrame(X))
    def cat_imputer(X):
        print(X.shape)
        return(imputer_cat.fit_transform(X))
        #return X.apply(lambda col: imputer_cat.fit_transform(col))  
    def one_hot_encode(X):
        print("one hot encode")
        ohe = OneHotEncoder(handle_unknown = 'ignore')
        #print("DF:: ")
        ohe.fit(pd.DataFrame(X))
        global hi
        hi=ohe
        #dump(ohe, 'onehot.joblib') 
        return ohe.transform(pd.DataFrame(X)).toarray()
    def min_max_scaling(X):
        scaler = MinMaxScaler()
        scaler.fit(X)
        global scale
        scale=scaler
        return scaler.transform(pd.DataFrame(X)).tolist()
    log_transform_pipeline = Pipeline([
    ('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
    ('imputer', SimpleImputer(strategy='mean')),   
    ('log_transform', FunctionTransformer(np.log1p))
    ])

    num_cols_pipeline = Pipeline([
    ('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
    ('imputer', SimpleImputer(strategy='mean')),
    ('min_max_scaler', FunctionTransformer(min_max_scaling, validate=False))
    ])

    cat_cols_pipeline = Pipeline([
    ('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
    ('imputer', SimpleImputer(strategy="most_frequent")),
    #    ('get_dummies', FunctionTransformer(get_dummies, validate=False))
    ('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))
    ])       

    steps_ = FeatureUnion([
    ('log_transform', log_transform_pipeline),
    ('num_cols', num_cols_pipeline),
    ('cat_cols', cat_cols_pipeline)
    ])
    full_pipeline = Pipeline([('steps_', steps_)])
    y = train['income'].map({'<=50K': 0, '>50K': 1})
    X = full_pipeline.fit_transform(train)
    z=train.index
    model = AdaBoostClassifier(n_estimators=300)
    X_train, X_test, y_train, y_test,z_train,z_test = train_test_split(X, y,z,shuffle=False)
    model = model.fit(X_train, y_train) 
    return model,X_test, y_test,y_train,hi,z_train,z_test,scale

In [61]:
train=get_dataset()

df is created Index(['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')
Index(['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')


In [62]:
train.head(5)

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [63]:
model,X_test, y_test, y_train,ohe,z_train,z_test,minmax_scaling = train_model(train)

one hot encode


In [64]:
z_train

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            33906, 33907, 33908, 33909, 33910, 33911, 33912, 33913, 33914,
            33915],
           dtype='int64', length=33916)

In [65]:
z_test[13]

33929

In [66]:
X_test[13]


array([0.        , 8.94754602, 0.56164384, 1.        , 0.07688077,
       0.        , 0.5       , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [69]:
minmax_scaling

MinMaxScaler()

In [67]:
key = train.iloc[33929].index
val = train.iloc[33929].values
dict_val = {i: j for i,j in zip(key,val)}
dict_val.pop('income')
X_Prod = pd.DataFrame(dict_val,index=[0])

In [68]:
X_Prod

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,58,State-gov,Doctorate,16.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,7688.0,0.0,50.0,United-States


In [70]:
model.predict_proba(X_test[13].reshape(1,-1))

array([[0.49231958, 0.50768042]])

In [71]:
y_test.shape

(11306,)

In [72]:
bye =model.predict(X_test)

In [73]:
bye=pd.Series(bye)

In [45]:
list(bye.where(bye == 1).dropna().index)

[13,
 14,
 16,
 19,
 20,
 36,
 37,
 43,
 51,
 57,
 69,
 73,
 75,
 79,
 93,
 101,
 102,
 103,
 105,
 108,
 115,
 118,
 128,
 132,
 141,
 142,
 145,
 148,
 150,
 151,
 167,
 189,
 200,
 202,
 204,
 208,
 210,
 215,
 227,
 232,
 239,
 242,
 246,
 264,
 266,
 283,
 294,
 299,
 301,
 302,
 304,
 310,
 315,
 328,
 342,
 351,
 358,
 360,
 361,
 368,
 369,
 381,
 384,
 385,
 396,
 398,
 400,
 408,
 415,
 421,
 429,
 433,
 439,
 446,
 454,
 457,
 472,
 473,
 478,
 489,
 500,
 510,
 512,
 515,
 521,
 522,
 523,
 530,
 533,
 537,
 540,
 544,
 551,
 555,
 558,
 569,
 570,
 578,
 585,
 595,
 599,
 611,
 613,
 622,
 625,
 627,
 631,
 654,
 660,
 661,
 664,
 667,
 668,
 669,
 670,
 672,
 676,
 688,
 697,
 703,
 712,
 715,
 720,
 723,
 726,
 731,
 732,
 733,
 734,
 735,
 739,
 744,
 745,
 755,
 758,
 765,
 767,
 772,
 781,
 788,
 790,
 791,
 797,
 798,
 802,
 810,
 812,
 815,
 817,
 818,
 822,
 825,
 834,
 839,
 845,
 846,
 858,
 861,
 862,
 864,
 865,
 869,
 871,
 875,
 885,
 888,
 902,
 903,
 911,
 

In [74]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, model.predict(X_test)))


0.8641429329559526


In [75]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 1, 0, 1])

In [76]:
from sklearn import metrics
confusion = metrics.confusion_matrix(y_test, y_pred)

In [77]:
confusion

array([[7991,  541],
       [ 995, 1779]])

In [78]:
(pd.DataFrame(y_pred)).value_counts()

0    8986
1    2320
dtype: int64

In [79]:

def test_model(train: pd.DataFrame,ohe:OneHotEncoder,scaler:MinMaxScaler) -> Tuple[AdaBoostClassifier,OneHotEncoder]:
    num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loss', 'hours-per-week']
    cat_cols = ['workclass', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']
    log_transform_cols = ['capital-loss', 'capital-gain']    
    def get_cat_cols(X):
        return X[cat_cols]
    def get_num_cols(X):
        return X[num_cols]
    def get_log_transform_cols(X):
        return X[log_transform_cols]
    def get_dummies(X):
        print('\n \n',type(X))
        return pd.get_dummies(pd.DataFrame(X))
    def cat_imputer(X):
        print(X.shape)
        return(imputer_cat.fit_transform(X))
        #return X.apply(lambda col: imputer_cat.fit_transform(col))  
    def one_hot_encode(X):
        print("one hot encode")
        #dump(ohe, 'onehot.joblib') 
        print(X)
        return ohe.transform(pd.DataFrame(X)).toarray()
    def min_max_scaling(X):
        return scaler.transform(pd.DataFrame(X)).tolist()

    log_transform_pipeline = Pipeline([
    ('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
    ('imputer', SimpleImputer(strategy='mean')),   
    ('log_transform', FunctionTransformer(np.log1p))
    ])

    num_cols_pipeline = Pipeline([
    ('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
    ('imputer', SimpleImputer(strategy='mean')),
    ('min_max_scaler', FunctionTransformer(min_max_scaling, validate=False))
    ])

    cat_cols_pipeline = Pipeline([
    ('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
    ('imputer', SimpleImputer(strategy="most_frequent")),
    #    ('get_dummies', FunctionTransformer(get_dummies, validate=False))
    ('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))
    ])       

    steps_ = FeatureUnion([
    ('log_transform', log_transform_pipeline),
    ('num_cols', num_cols_pipeline),
    ('cat_cols', cat_cols_pipeline)
    ])
    full_pipeline = Pipeline([('steps_', steps_)])
    X = full_pipeline.fit_transform(train)
   
    return X

In [80]:
bye

0        0
1        0
2        0
3        0
4        0
        ..
11301    0
11302    0
11303    1
11304    0
11305    1
Length: 11306, dtype: int64

In [81]:
x_prod = test_model(X_Prod,ohe,minmax_scaling)

one hot encode
[[' State-gov' ' Married-civ-spouse' ' Prof-specialty' ' Husband'
  ' White' ' Male' ' United-States']]


In [82]:
x_prod

array([[0.        , 8.94754602, 0.56164384, 1.        , 0.07688077,
        0.        , 0.5       , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [83]:

model.predict_proba(x_prod)

array([[0.49231958, 0.50768042]])

In [45]:
y_pred = model.predict_proba(x_prod)

In [46]:
y_pred

array([[0.56193188, 0.43806812]])

In [56]:
train[train['income']=='>50K']

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
7,52,Self-emp-not-inc,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,45.0,United-States,>50K
8,31,Private,Masters,14.0,Never-married,Prof-specialty,Not-in-family,White,Female,14084.0,0.0,50.0,United-States,>50K
9,42,Private,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178.0,0.0,40.0,United-States,>50K
10,37,Private,Some-college,10.0,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0.0,0.0,80.0,United-States,>50K
11,30,State-gov,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0.0,0.0,40.0,India,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45195,60,Private,Assoc-voc,11.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,7688.0,0.0,40.0,United-States,>50K
45197,38,Private,Masters,14.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,50.0,United-States,>50K
45198,43,Local-gov,Masters,14.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,1902.0,50.0,United-States,>50K
45204,40,Private,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,15024.0,0.0,55.0,United-States,>50K


In [115]:
index_one = train[train['income']=='>50K'].index.to_list()

In [62]:
key = train[train['income']=='>50K'].iloc[9].index.to_list()

val = list(train[train['income']=='>50K'].iloc[9].values)

In [63]:
key = train[train['income']=='>50K'].iloc[9].index.to_list()
val = list(train[train['income']=='>50K'].iloc[9].values)
dict_val = {i: j for i,j in zip(key,val)}
dict_val.pop('income')
X_Prod = pd.DataFrame(dict_val,index=[0])

In [64]:
dict_val

{'age': 47,
 'workclass': ' Private',
 'education_level': ' Prof-school',
 'education-num': 15.0,
 'marital-status': ' Married-civ-spouse',
 'occupation': ' Prof-specialty',
 'relationship': ' Wife',
 'race': ' White',
 'sex': ' Female',
 'capital-gain': 0.0,
 'capital-loss': 1902.0,
 'hours-per-week': 60.0,
 'native-country': ' Honduras',
 'income': '>50K'}

'>50K'

In [66]:
X_Prod = pd.DataFrame(dict_val,index=[0])

In [67]:
X_Prod

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,47,Private,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Wife,White,Female,0.0,1902.0,60.0,Honduras


In [33]:
x_prod = train_model(X_Prod,ohe)

one hot encode
[[' State-gov' ' Married-civ-spouse' ' Prof-specialty' ' Husband'
  ' White' ' Male' ' United-States']]


In [34]:
x_prod = x_prod.reshape(1,-1)

In [35]:
x_prod

<1x89 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [71]:
y_pred = model.predict_proba(x_prod)

In [72]:
y_pred

array([[0.56347513, 0.43652487]])

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np


In [8]:
n_samples, n_features, n_classes = 10, 2, 2
data = train  # 10 training examples
labels = np.random.randint(n_classes, size=n_samples)  # 10 labels
indices = np.arange(n_samples)

In [11]:
indices

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
(
    data_train,
    data_test,
    labels_train,
    labels_test,
    indices_train,
    indices_test,
) = train_test_split(data, labels, indices, test_size=0.2)

In [36]:
import pandas as pa
dicst_temp={'col1':[1,2,3,4],'col2':['a','b','c','d'],'col3':[2,4,None,None],'col4':'str','col5':(1,7,3,4),'col6':1}
df=pa.DataFrame(dicst_temp)

In [37]:
df

,col1,col2,col3,col4,col5,col6
0,1,a,2.0,str,1,1
1,2,b,4.0,str,7,1
2,3,c,NaN,str,3,1
3,4,d,NaN,str,4,1


In [61]:
df.isna()

,col1,col2,col3,col4,col5,col6
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,True,False,False,False
3,False,False,True,False,False,False


In [69]:
for i in list(df.columns):
    print(df[df[i].isna()][i].index)

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([2, 3], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')


In [51]:
import math
def check_nan(val):
    if type(val)!='str':
        if math.isnan(val):
            print(val)

In [52]:
df.applymap(check_nan)

TypeError: must be real number, not str